<img src="images/steno3d.png" width="200"></img>

<br>

# Steno3D Resources

In this notebook, we will explore the different types of data and resources supported by <a href="https://steno3d.com">Steno3D</a>. An overview is also available in the <a href="https://steno3d.readthedocs.io/en/latest/content/what_is_steno3d.html">online documentation</a>. Specific topics include:

- <a href="#Projects">Projects</a>
- <a href="#Resource-Components">Resource Components</a>
- Resource Creation:
    - <a href="#Point-Resources">Points</a>
    - <a href="#Line-Resources">Lines</a>
    - <a href="#Surface-Resources">Surfaces</a>
    - <a href="#Volume-Resources">Volumes</a>

___
**Navigation**
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___

## Initialization

Importing modules, logging in and defining helper functions must be done prior to constructing any resource in this journal.

In [ ]:
import steno3d
from steno3d.examples import Images

import numpy as np
from properties.vmath import Vector
from scipy.spatial import Delaunay

#### Login to Steno3D

Before using Steno3D, you need to login. You can <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key.

In [ ]:
steno3d.login()

Here we initialize some spatial variables and simple helper functions used throughout the notebook

In [ ]:
x = [0., -250., 450.]
y = [1000., 0., 150.]
z = [0., 0., 0.]
r = [500., 400., 300.]

def cluster_coords(x, y, z, rad, npts):
    """Return a cluster of points centered
    around x, y, z within radius rad
    
    Input:  x, y, z - center of points
            rad - approximate containing radius
            npts - number of points output
    
    Output: npts x 3 array of points
    """
    return np.r_[
        [np.random.normal(x, rad/4., npts),
         np.random.normal(y, rad/4., npts),
         np.random.normal(z, rad/4., npts)]
    ].T

def dist_fcn(verts, origin):
    """Distance between origin and a list of points
    
    Input:  verts - list of points
            origin - one point
            
    Output: array of distances, length = len(verts)
    """
    pt_diff = verts-np.r_[verts.shape[0]*[origin]]
    return np.sqrt(np.sum(pt_diff * pt_diff, axis=1))

## Projects

The first step when working in Steno3D is creating a `Project`. Projects contain related resources; they can be viewed and shared on steno3d.com. The properties of projects are:
- `title`: Title of the project
- `description`: Description of the project
- `public`: If `True`, the project is viewable by anyone online. If `False`, the project is private and only viewable by people you choose to share it with. Free and Academic accounts have limits on the number of private projects allowed; if you would like more, upgrade your account at steno3d.com.
- `resources`: List of resources the project contains.

You can <a href="https://steno3d.com/explore">explore public projects on steno3d.com</a>. Additional <a href="https://python.steno3d.com/en/latest/content/api/projects.html">API documentation</a> is available online.

In [ ]:
my_proj = steno3d.Project(
    title='Resource Notebook Project',
    description='This project is from the Steno3D Resource notebook',
    public=True
)

## Resource Components

Before building resources in Steno3D, here is a brief overview of the different components you will need: `mesh`, `data`, and `textures`.

<img src="images/Steno3DAPI.png" width="750"></img>

### Mesh
The `mesh` defines the geometry of the resource. It contains vertices, faces, grid-cell spacing, etc. Each type of resource has one or more meshes that may be used, corresponding to the dimensionality of the resource. All resources require a mesh. Mesh API documentation is available within the corresponding <a href="http://python.steno3d.com/en/latest/content/api/resources/index.html">resource documentation</a>.
### Data
The `data` is simply an array of values that correspond to mesh locations. When binding data to a resource, the mesh location must be specified in conjunction with the data array. <a href="http://python.steno3d.com/en/latest/content/api/resources/data.html">Data API documetation</a> is available online.
### Textures
The `textures` are images that exist in space and are mapped to locations on the mesh. They do not correspond to certain mesh locations. <a href="http://python.steno3d.com/en/latest/content/api/resources/texture.html">Texture API documetation</a> is available online.

___
**Navigation**
- <a href="#Steno3D-Resources">Top</a>
- <a href="3.%20Example%20Wolf%20Pass%20Project.ipynb">Next notebook: 3. Example Wolf Pass Project</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___

## Point Resources

Points are 0-dimensional resources, for example, GPS waypoints, earthquake foci, or drone locations. The properties of `Point` are:
- `title`
- `description`
- `mesh` (required): Points use `Mesh0D`, a mesh that only contains `vertices`
- `data`: A list of bound `DataArray`s
- `textures`: A list of `Texture2DImage`s
- `opts`: A dictionary of options
- `project` (required): The project(s) that contain the resource

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">API documentation</a> is available online.
___
__Note:__ Before moving forward, please ensure that you completed the <a href="#Initialization">initialization steps</a> and made a <a href="#Projects">project</a> to contain your resource.

Let's start by creating a point. All resources (including points) must be initialized with their project; this gives the resource a context to start with. Resources can be added to and removed from projects retroactively as well, so the initial project isn't set in stone.

In [ ]:
my_point = steno3d.Point(
    project=my_proj,
    title='Basic Point Resource'
)

#### Point Mesh

Now we can use the `cluster_coords` function to generate vertices for a mesh. `Mesh0D` have properties:
- `vertices` (required): 3 x n array of spatial coordinates
- `title`
- `description`

In [ ]:
my_verts=cluster_coords(x=x[0], y=y[0], z=z[0], rad=r[0], npts=100)
my_mesh = steno3d.Mesh0D(
    vertices=my_verts
)

This mesh must be added to the point resource:

In [ ]:
my_point.mesh = my_mesh

#### Binding Point Data

We now need to bind data to the point resource. In Steno3D, binding data to a resource is somewhat confusing, so follow along carefully. You may also refer to the <a href="http://python.steno3d.com/en/latest/content/api/resources/data.html">API documentation</a>. First we need a `DataArray` that contains:
- `array` (required): 1D-array of data
- `title`
- `description`

Our first data set will be distance from the origin:

In [ ]:
origin_dist=dist_fcn(my_verts, [0, 0, 0])
my_data = steno3d.DataArray(
    title='Distance from Origin',
    array=origin_dist
)

We then must bind this data to specific mesh locations. This is done with a dictionary that has two entries:
- `location` (required): For point resources, the only `location` available is 'N' for nodes. Other types of resources allow `location` to be 'N' or 'CC', nodes or cell centers
- `data` (required): a DataArray

In [ ]:
my_point.data = dict(
    data=my_data,
    location='N'
)

Resources may have any number of associated data arrays. For multiple data sets, you can use a list of these `location`/`data` dictionaries. Here, we will generate a second data set and bind both data sets to the point resource.

In [ ]:
cluster_dist = dist_fcn(my_verts, [x[0], y[0], z[0]])
my_other_data = steno3d.DataArray(
    title='Distance from Cluster Center',
    array=cluster_dist
)
my_point.data = [
    dict(
        location='N',
        data=my_data
    ),
    dict(
        location='N',
        data=my_other_data
    )
]

#### Point Textures

Finally, we can add textures to point resources. `Texture2DImage`s map an image in space. They contain:
- `image` (required): a png image to map
- `O` (required): the origin vector - this sets the lower-left corner of the image in space
- `U` and `V` (required): the axes vectors - these determine the length and the direction of the sides of the image relative to `O`

For additional explanation about how the texture mapping works, please check out the <a href="http://python.steno3d.com/en/latest/content/api/resources/texture.html">API documentation</a>. Here, we will load up an example image file and map it to the center of our point cluser.

In [ ]:
wood_img = Images.fetch_data(filename='woodplanks.png', verbose=False)
my_tex = steno3d.Texture2DImage(
    image=wood_img,
    O=[-250., 750., 0],
    U=[500., 0., 0.],
    V=[0., 500., 0.]
)

We can now add this texture to our point. If we have multiple textures for a resource, we can simply add them as a list.

In [ ]:
my_point.textures = my_tex

#### Point Options

The last thing to note are the options we can set. For now, this includes `color` and `opacity`. These are set using a dictionary:

In [ ]:
my_point.opts = dict(
    color='black',
    opacity=1
)

#### Point Validation, Uploading, and Plotting

That concludes the creation of our first Steno3D resource. To ensure that there are not any problems, you can call `validate()`. This checks that all aspects of the resource are built correctly.

In [ ]:
my_point.validate()

We can also `upload` resources or projects to <a href="https://steno3d.com">steno3d.com</a> at any time as long as they are valid. Calling `upload()` on a resource actually uploads its containing projects. You may set `sync=True` if you would like to push any future changes to <a href="https://steno3d.com">steno3d.com</a>. We are going to upload our point resource but keep syncing off for now.

In [ ]:
my_point.upload()

Once resources are uploaded, you can `plot` them to make sure it looks how you expect. You can also view the entire project online at <a href="https://steno3d.com">steno3d.com</a> at the project url.

In [ ]:
my_point.plot()

In [ ]:
print(my_proj.url)

Let's make two more point resources in our project; all the above steps will be condensed down.

In [ ]:
my_mesh_b = steno3d.Mesh0D(
    vertices=cluster_coords(x=x[1], y=y[1], z=z[1], rad=r[1], npts=75)
)
my_data = steno3d.DataArray(
    title='Distance from Origin',
    array=dist_fcn(my_mesh_b.vertices, [0., 0., 0.])
)
my_other_data = steno3d.DataArray(
    title='Distance from Cluster Center',
    array=dist_fcn(my_mesh_b.vertices, [x[1], y[1], z[1]])
)
my_tex = steno3d.Texture2DImage(
    image=Images.fetch_data(filename='metal.png', verbose=False),
    O=[-350., 0., -100.],
    U=[200., 0., 0.],
    V=[0., 0., 200.]
)
my_opts = dict(
    color='#46B742'
)
another_point = steno3d.Point(
    project=my_proj,
    title='Green Points',
    mesh=my_mesh_b,
    data=[dict(location='N', data=my_data),
          dict(location='N', data=my_other_data)],
    textures=[my_tex],
    opts=my_opts
)

In [ ]:
my_verts=cluster_coords(x=x[2], y=y[2], z=z[2], rad=r[2], npts=50)
and_another_point = steno3d.Point(
    project=my_proj,
    title='More Points',
    mesh=steno3d.Mesh0D(
        vertices=my_verts
    ),
    data=[
        dict(
            location='N',
            data=steno3d.DataArray(
                title='Distance from Origin',
                array=dist_fcn(my_verts, [0., 0., 0.])
            )
        ),
        dict(
            location='N',
            data=steno3d.DataArray(
                title='Distance from Cluster Center',
                array=dist_fcn(my_verts, [x[2], y[2], z[2]])
            )
        )
    ],
    opts=dict(
        color=[123, 124, 125]
    )
)

To upload both these resources, you can upload the containing project. That will upload any changes, including new resources. Visit the project page again to see the new resources added.

In [ ]:
my_proj.upload()

___
**Navigation**
- <a href="#Steno3D-Resources">Top of page</a>
- <a href="3.%20Example%20Wolf%20Pass%20Project.ipynb">Next notebook: 3. Example Wolf Pass Project</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___

## Line Resources

Lines are 1-dimensional resources, for example, boreholes, tectonic plate motion directions, or drone flight paths. The properties of `Line` are:
- `title`
- `description`
- `mesh` (required): Lines use `Mesh1D`, a mesh that contains `vertices` and `segments`
- `data`: A list of bound `DataArray`s
- `opts`: A dictionary of options
- `project` (required): The project(s) that contain the resource

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">API documentation</a> is available online.
___
__Note:__ Before moving forward, please ensure that you completed the <a href="#Initialization">initialization steps</a> and made a <a href="#Projects">project</a> to contain your resource.

Just like the <a href="#Point-Resources">points</a>, we need to provide a project on initialization of lines:

In [ ]:
my_line = steno3d.Line(
    project=my_proj,
    title='Basic Line Resource'
)

#### Line Mesh

The meshes used with lines are `Mesh1D`. These meshes have properties:
- `vertices` (required): 3 x n array of spatial coordinates
- `segments` (required): 2 x m array of vertex indices connecting each segment
- `title`
- `description`

First we will construct the geometry, 3 vertical lines, then we will build the mesh.

In [ ]:
my_verts = np.c_[
    np.r_[x[0]*np.ones(10), x[1]*np.ones(10), x[2]*np.ones(10)],
    np.r_[y[0]*np.ones(10), y[1]*np.ones(10), y[2]*np.ones(10)],
    np.r_[np.linspace(z[0]-r[0], -700, 10), np.linspace(z[1]-r[1], -700, 10), np.linspace(z[2]-r[2], -700, 10)]
]
my_segs = np.c_[
    np.r_[range(0, 9), range(10, 19), range(20, 29)],
    np.r_[range(1, 10), range(11, 20), range(21, 30)]
]

In [ ]:
my_mesh = steno3d.Mesh1D(
    vertices=my_verts,
    segments=my_segs
)
my_line.mesh = my_mesh

#### Binding Line Data

We now need to bind data to the line resource. For a bit more detail, see the <a href="#Binding-Point-Data">point data</a> section. In a nutshell, we need a `DataArray` that contains:
- `array` (required): 1D-array of data
- `title`
- `description`

Then we need to bind this to our line with a dictionary containing:
- `location` (required): For lines, this is 'N' (nodes or vertices) or 'CC' (cell centers or segments)
- `data` (required): the DataArray

More than one data set can be bound to a line by using a list of these dictionaries.

For our example line, we will just create some data using `dist_fcn`:

In [ ]:
my_data_0 = steno3d.DataArray(
    array=dist_fcn(my_verts, [x[0], y[0], x[0]]),
    title='First Distance'
)
my_data_1 = steno3d.DataArray(
    array=dist_fcn(my_verts, [x[1], y[1], x[1]]),
    title='Second Distance'
)
my_data_2 = steno3d.DataArray(
    array=dist_fcn(my_verts, [x[2], y[2], x[2]]),
    title='Third Distance'
)

All of these are defined on the vertices, so we will bind them to 'N' location:

In [ ]:
my_line.data = [
    dict(
        location='N',
        data=my_data_0
    ),
    dict(
        location='N',
        data=my_data_1
    ),
    dict(
        location='N',
        data=my_data_2
    )
]

#### Line Options

Like the <a href="#Point-Options">point options</a>, you can assign `color` and `opacity` as opts using a dictionary.

In [ ]:
my_line.opts = dict(
    color='red',
    opacity=0.25
)

#### Line Validation, Uploading, and Plotting

Lines, like <a href="#Point-Validation,-Uploading,-and-Plotting">points</a>, have validation, uploading, and plotting:

In [ ]:
my_line.validate() # This is also called on upload.
my_line.upload() # Uploading the line uploads the containing project.
my_line.plot() # This just plots the line.

You can return to the project online to see this line added.

In [ ]:
print(my_proj.url)

___
**Navigation**
- <a href="#Steno3D-Resources">Top of page</a>
- <a href="3.%20Example%20Wolf%20Pass%20Project.ipynb">Next notebook: 3. Example Wolf Pass Project</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___

## Surface Resources

Surfaces are 2-dimensional resources, for example, topography or geologic unit boundaries. The properties of `Surface` are:
- `title`
- `description`
- `mesh` (required): Surfaces may use `Mesh2D` or `Mesh2DGrid`
- `data`: A list of bound `DataArray`s
- `textures`: A list of `Texture2DImage`s
- `opts`: A dictionary of options
- `project` (required): The project(s) that contain the resource

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html">API documentation</a> is available online.
___
__Note:__ Before moving forward, please ensure that you completed the <a href="#Initialization">initialization steps</a> and made a <a href="#Projects">project</a> to contain your resource.

Initialize just like the <a href="#Point-Resources">points</a>:

In [ ]:
my_surface = steno3d.Surface(
    project=my_proj,
    title='Basic Surface Resource'
)

#### Surface Mesh2D

There are two types of surface meshes available. The first is `Mesh2D`. This is very similar to the <a href="#Line-Mesh">line mesh</a> except it has triangles instead of segments and adds options:
- `vertices` (required): 3 x n array of spatial coordinates
- `triangles` (required): 3 x m array of vertex indices connecting each triangular face
- `title`
- `description`
- `opts`: The only available option is `wireframe`, set to `True` or `False` with a dictionary 

The other type of surface mesh is <a href="#Surface-Mesh2DGrid">Mesh2DGrid</a>, described below.

Let's build the components of our `Mesh2D`. We will generate some random points on a sphere, then use Delaunay triangulation to construct a convex hull around them.

In [ ]:
nverts = 50
my_verts = Vector(np.c_[
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5
])
my_verts = np.array(my_verts.normalize())*r[0]
my_verts = my_verts + np.r_[my_verts.shape[0]*[[x[0], y[0], z[0]]]]
my_tris = Delaunay(my_verts).convex_hull

Now construct the mesh with these components:

In [ ]:
my_mesh = steno3d.Mesh2D(
    vertices=my_verts,
    triangles=my_tris
)

We can make the mesh display its wire frame by specifying `wireframe=True` in the options:

In [ ]:
my_mesh.opts = dict(
    wireframe=True
)

And add this mesh to the surface:

In [ ]:
my_surface.mesh = my_mesh

#### Surface Mesh2DGrid

The other type of surface mesh is `Mesh2DGrid`. This is defined differently than the meshes so far:
- `h1` and `h2` (required): x and y grid spacings
- `x0`: origin vector corresponding to the lower-left corner of the grid
- `Z`: mesh vertex topography
- `title`
- `description`
- `opts`: The only available option is `wireframe`, set to `True` or `False` with a dictionary

We have to create a new surface to use this mesh since each surface may only have one mesh.

In [ ]:
my_grid_surf = steno3d.Surface(
    project=my_proj,
    title='Basic surface resource grid'
)

In [ ]:
x_space = np.linspace(-300., 300., num=60)
y_space = np.linspace(-750., -440., num=31)

h1 = np.diff(x_space)
h2 = np.diff(y_space)
x0 = [-300., -750., 0.]

X, Y = np.meshgrid(x_space, y_space)
topo = lambda X, Y: 25.*np.sin(X/30.+Y/100.)
Z = topo(X, Y).flatten(order='C')

In [ ]:
my_grid_mesh = steno3d.Mesh2DGrid(
    h1=h1,
    h2=h2,
    x0=x0,
    Z=Z
)

my_grid_surf.mesh = my_grid_mesh

#### Binding Surface Data

We now need to bind data to the surface resource. For a bit more detail, see the <a href="#Binding-Point-Data">point data</a> section. In a nutshell, we need a `DataArray` that contains:
- `array` (required): 1D-array of data
- `title`
- `description`

Then we need to bind this to our surface with a dictionary containing:
- `location` (required): For surfaces, this is 'N' (nodes or vertices) or 'CC' (cell centers or faces/triangles)
- `data` (required): the DataArray

More than one data set can be bound to a surface by using a list of these dictionaries.

For our example surface, we will just create some random data.

In [ ]:
my_data = steno3d.DataArray(
    title='Random Face Data',
    array=np.random.rand(my_grid_mesh.nC)
)

my_grid_surf.data = dict(
    location='CC',
    data=my_data
)

#### Surface Textures

We can add textures to surface resources, like <a href="Point-Textures">points</a>. `Texture2DImage`s map an image in space. They contain:
- `image` (required): a png image to map
- `O` (required): the origin vector - this sets the lower-left corner of the image in space
- `U` and `V` (required): the axes vectors - these determine the length and the direction of the sides of the image relative to `O`

Here, we will load up an example image file and map it to the center of our surface grid cluser.

In [ ]:
steno3d_img = Images.fetch_data(filename='steno3d_logo_text.png', verbose=False)
my_tex = steno3d.Texture2DImage(
    image=steno3d_img,
    O=[-300., -750., 0],
    U=[600., 0., 0.],
    V=[0., 310., 0.]
)

my_grid_surf.textures = my_tex

#### Surface Options

Like the <a href="#Point-Options">point options</a>, you can assign `color` and `opacity` as opts using a dictionary.

In [ ]:
my_surface.opts = dict(
    color='black',
    opacity=0.5
)
my_grid_surf.opts = dict(
    color='white',
    opacity=1
)

#### Surface Validation, Uploading, and Plotting

Surfaces, like <a href="#Point-Validation,-Uploading,-and-Plotting">points</a>, have validation, uploading, and plotting. We are not going to call validate, because it is called on upload.

In [ ]:
my_proj.upload()
my_surface.plot()
my_grid_surf.plot()

Let's make a couple more surfaces and add them to our project:

In [ ]:
nverts = 40
my_verts = Vector(np.c_[
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5
])
my_verts = np.array(my_verts.normalize())*r[1]
my_verts = my_verts + np.r_[my_verts.shape[0]*[[x[1], y[1], z[1]]]]
my_tris = Delaunay(my_verts).convex_hull

my_surf_2 = steno3d.Surface(
    project=my_proj,
    mesh=steno3d.Mesh2D(
        vertices=my_verts,
        triangles=my_tris,
        opts=dict(wireframe=True)
    ),
    opts=dict(
        color='#46B742',
        opacity=0.5
    )
)
my_surf_2.upload(sync=False)

nverts = 30
my_verts = Vector(np.c_[
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5,
    np.random.rand(nverts)-.5
])
my_verts = np.array(my_verts.normalize())*r[2]
my_verts = my_verts + np.r_[my_verts.shape[0]*[[x[2], y[2], z[2]]]]
my_tris = Delaunay(my_verts).convex_hull

my_surf_3 = steno3d.Surface(
    project=my_proj,
    mesh=steno3d.Mesh2D(
        vertices=my_verts,
        triangles=my_tris,
        opts=dict(wireframe=True)
    ),
    opts=dict(
        color=[123, 124, 125],
        opacity=0.5
    )
)
my_surf_3.upload(sync=False)

print(my_proj.url)

___
**Navigation**
- <a href="#Steno3D-Resources">Top of page</a>
- <a href="3.%20Example%20Wolf%20Pass%20Project.ipynb">Next notebook: 3. Example Wolf Pass Project</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___

## Volume Resources

Volumes are 3-dimensional resources; they are continuous measurements in space like rock type, temperature, and electrical resistivity. The properties of `Volume` are:
- `title`
- `description`
- `mesh` (required): Volumes use `Mesh3DGrid`
- `data` (required): A list of bound `DataArray`s
- `opts`: A dictionary of options
- `project` (required): The project(s) that contain the resource

One significant difference between volumes and the other resources is that data is required. Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/volume.html">API documentation</a> is available online.
___
__Note:__ Before moving forward, please ensure that you completed the <a href="#Initialization">initialization steps</a> and made a <a href="#Projects">project</a> to contain your resource.

In [ ]:
my_volume = steno3d.Volume(
    project=my_proj,
    title='Basic Volume Resource'
)

#### Volume Mesh3DGrid

Volume meshes are `Mesh3DGrid`s. These are similar to <a href="#Surface-Mesh2DGrid">Mesh2DGrids</a>:
- `h1`, `h2`, and `h3` (required): x, y, and z grid spacings
- `x0`: origin vector corresponding to the lower-left corner of the grid
- `title`
- `description`
- `opts`: The only available option is `wireframe`, set to `True` or `False` with a dictionary

In [ ]:
x_space = np.linspace(-700., 800., num=50)
y_space = np.linspace(-440., 1540., num=75)
z_space = np.linspace(-650., 0., num=25)

h1 = np.diff(x_space)
h2 = np.diff(y_space)
h3 = np.diff(z_space)
x0 = [-700., -440., -650.]

In [ ]:
my_mesh = steno3d.Mesh3DGrid(
    h1=h1,
    h2=h2,
    h3=h3,
    x0=x0
)

my_mesh.opts = dict(
    wireframe=False
)

my_volume.mesh = my_mesh

#### Binding Volume Data

We now need to bind data to the volume resource. For a bit more detail, see the <a href="#Binding-Point-Data">point data</a> section. In a nutshell, we need a `DataArray` that contains:
- `array` (required): 1D-array of data
- `title`
- `description`

Then we need to bind this to our volume with a dictionary containing:
- `location` (required): For volumes, this must be 'CC' (cell centers)
- `data` (required): the DataArray

More than one data set can be bound to a volume by using a list of these dictionaries.

Let's generate several sets of distance data and add them to our volume.

In [ ]:
x_cent = (x_space[:-1] + x_space[1:])/2.
y_cent = (y_space[:-1] + y_space[1:])/2.
z_cent = (z_space[:-1] + z_space[1:])/2.
X, Y, Z = np.meshgrid(y_cent, z_cent, x_cent) # Axis swapping is required to keep the flattened order correct
X = X.flatten(order='C')
Y = Y.flatten(order='C')
Z = Z.flatten(order='C')

dist0 = dist_fcn(np.r_[[X, Y, Z]].T, [y[0], z[0], x[0]])
dist0 = np.array([d if d > r[0] else np.nan for d in dist0]) # nan values are not plotted in Steno3D

dist1 = dist_fcn(np.r_[[X, Y, Z]].T, [y[1], z[1], x[1]])
dist1 = np.array([d if d > r[1] else np.nan for d in dist1])

dist2 = dist_fcn(np.r_[[X, Y, Z]].T, [y[2], z[2], x[2]])
dist2 = np.array([d if d > r[2] else np.nan for d in dist2])

dist = (dist0 + dist1 + dist2)

In [ ]:
my_volume.data = [
    dict(
        location='CC',
        data=steno3d.DataArray(
            array=dist,
            title='Distances from all points'
        )
    ),
    dict(
        location='CC',
        data=steno3d.DataArray(
            array=dist0,
            title='Distance from one point'
        )
    ),
    dict(
        location='CC',
        data=steno3d.DataArray(
            array=dist1,
            title='Distance from another point'
        )
    ),
    dict(
        location='CC',
        data=steno3d.DataArray(
            array=dist2,
            title='Distance from a final point'
        )
    )
]

#### Volume Options

Like the <a href="#Point-Options">point options</a>, you can assign `color` and `opacity` as opts using a dictionary.

In [ ]:
my_volume.opts = dict(
    color = 'black',
    opacity = 1
)

#### Volume Validation, Uploading, and Plotting

Volumes are validated on upload and can be plotted the same way as <a href="#Point-Validation,-Uploading,-and-Plotting">points</a>.

In [ ]:
my_volume.upload()
my_volume.plot()

Now that we are done constructing a project, explore on the <a href="https://steno3d.com">Steno3D website</a> and share with your colleagues!

In [ ]:
print(my_proj.url)

## Want to work with *your* data?

You can download Steno3D from pip:
```
pip install steno3d
```
or install from source
```
git clone https://github.com/3ptscience/steno3dpy.git
python setup.py install 
```

___
**Navigation**
- <a href="#Steno3D-Resources">Top of page</a>
- <a href="3.%20Example%20Wolf%20Pass%20Project.ipynb">Next notebook: 3. Example Wolf Pass Project</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/3ptscience/steno3dpy-notebooks/issues/new">Report an issue</a>
___